# `montecarlo.py`

This notebook tests the `montecarlo.py` module.

This module contains several methods designed to evaluate the probability of an event to be assigned to a component of the mixture and to evaluate the expected value of mean and covariance matrix of each of these components, both conditioned on the events already assigned to the component.

## Utilities

First of all, we will test some utility methods used by the main methods.

### `_log_add`

Numerically stable method to compute $\log\big(e^x + e^y\big)$. Works with doubles only.

In [ ]:
import numpy as np
from figaro.montecarlo import _log_add

x = np.array([0, 1, -np.inf, 1, 0])
y = np.array([1, 0, 1, -np.inf, 0])

for xi, yi in zip(x,y):
    print("log_sum = {0}, expected = {1}".format(_log_add(xi, yi), np.log(np.exp(xi) + np.exp(yi))))

### `_log_add_array`

Vectorised version of `log_add`. Works with arrays only, shape must match.

In [ ]:
from figaro.montecarlo import _log_add_array

z = _log_add_array(x,y)

for zi, zi_exp in zip(z, np.log(np.exp(x) + np.exp(y))):
    print("log_sum = {0}, expected = {1}".format(zi, zi_exp))

If this is not the case, if `len(x) > len(y)`, returns an array with the same lenght as `x`. Otherwise, the behaviour is undefined (no checks implemented).

In [ ]:
print('len(x)<len(y): {0}'.format(_log_add_array(x[:3], y)))
print('len(x)>len(y): {0}'.format(_log_add_array(x, np.zeros(1))))

### `log_norm_1d`

Log PDF of 1-dimensional Gaussian distribution, to be compared with Scipy's `norm().logpdf`. Works with doubles.

In [ ]:
from figaro.montecarlo import log_norm_1d
from scipy.stats import norm

mu  = 2
var = 0.1
x   = np.linspace(-5,5,1000)

logpdf_scipy = norm(mu, np.sqrt(var)).logpdf(x)
logpdf_figaro = np.array([log_norm_1d(xi, mu, var) for xi in x])

print(np.alltrue(logpdf_scipy == logpdf_figaro))

The element-wise comparison seems to fail: let's have a look at the differences.

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
a = ax.plot(x, logpdf_figaro - logpdf_scipy)

print(np.allclose(logpdf_scipy, logpdf_figaro, atol = 1e-13, rtol = 0))

### `inv_jit`


Numba-decorated `numpy.linalg.inv` method. Faster than pure Numpy on a single matrix.

In [ ]:
from figaro.montecarlo import inv_jit
from scipy.stats import invwishart

M = invwishart(3, np.identity(3)).rvs()

print('Numpy:')
%timeit np.linalg.inv(M)
print('Numba:')
%timeit inv_jit(M)

print(np.allclose(np.linalg.inv(M), inv_jit(M), atol = 1e-13))

### `logdet_jit`

Numba-decorated logarithm of `numpy.linalg.det` method. Faster than pure Numpy on a single matrix.

In [ ]:
from figaro.montecarlo import logdet_jit

M = invwishart(3, np.identity(3)).rvs()

print('Numpy:')
%timeit np.log(np.linalg.det(M))
print('Numba:')
%timeit logdet_jit(M)

print(np.allclose(np.log(np.linalg.det(M)), logdet_jit(M), atol = 1e-13))

### `scalar_product`

Given a matrix $M$ and a vector $v$, computes $\langle v, Mv\rangle$. Compared with `np.dot(v, np.dot(M, v))`.

In [ ]:
from figaro.montecarlo import scalar_product

M = invwishart(3, np.identity(3)).rvs()
v = np.random.uniform(size = 3)

print('Numpy:')
%timeit np.dot(v, np.dot(M, v))
print('Numba:')
%timeit scalar_product(v, M, len(v))

print(np.alltrue(scalar_product(v, M, len(v)) == np.dot(v, np.dot(M, v))))

### `log_norm`

Log PDF of N-dimensional Gaussian distribution, to be compared with Scipy's `multivariate_normal().logpdf`. Works with doubles.

In [ ]:
from figaro.montecarlo import log_norm
from scipy.stats import multivariate_normal

mu  = np.ones(2)
var = np.identity(2)*0.1**2
x   = np.linspace(0,2,100)
y   = np.linspace(0,2,100)

z = []
for xi in x:
    for yi in y:
        z.append(np.array([xi,yi]))
z = np.array(z)

logpdf_scipy  = multivariate_normal(mu, var).logpdf(z)
logpdf_figaro = np.array([log_norm(zi, mu, var) for zi in z])

print(np.alltrue(logpdf_scipy == logpdf_figaro))

The element-wise comparison seems to fail: let's have a look at the differences.

In [ ]:
fig, ax = plt.subplots()
a = ax.plot(logpdf_figaro - logpdf_scipy)

print(np.allclose(logpdf_scipy, logpdf_figaro, atol = 1e-13, rtol = 0))